# OpenAI SDK Agent
We will use openai-sdk-agents to create simple AI agents

Learn more about them in https://github.com/openai/openai-agents-python

Documentation: https://openai.github.io/openai-agents-python/

## Create .env file
- Create a .env file in your working directory
- Sign up in openai platform https://auth.openai.com/log-in
- Get the free tier API key from OPENAI, https://platform.openai.com/account/api-keys
- Add the key in .env file

In [ ]:
# OPENAI_API_KEY="your_openai_api_key_here"

## Load environment

In [ ]:
# Load the environment variables from a .env file
from dotenv import load_dotenv
load_dotenv(override=True)

## Import libraries

In [ ]:
# From the agents library import necessary components
from agents import Agent, Runner, trace
import os

## Create a basic Agent

In [ ]:

# Make an basic agent with name, instructions and model
# Other models from openai are gpt-4.1-mini.
# Find more on https://auth.openai.com/log-in
gpt_agent = Agent(name="Jokester", 
                instructions="You are a joke teller", 
                model="gpt-4.1-nano")


## Run Agent in non streaming mode

In [ ]:
# Use Runner.run(agent, prompt) is non streaming mode of running agent
# Since it is an async function, we need to use 'await' to call it
query = "Tell a joke about Software engineer who is an Autonomous AI Agents"
result = await Runner.run(gpt_agent, query)
print(result.final_output)

Logs can be traced at https://platform.openai.com/traces

Default trace name would be "Agent workflow"

## Use custom trace name

You can use with statement to give a custom name to trace.

In [ ]:
with trace("Telling a joke"):
    result = await Runner.run(gpt_agent, query)
    print(result.final_output)

## Use trace id

Alternatively, you can generate a trace id and use that.

In [ ]:
from agents import gen_trace_id

trace_id = gen_trace_id()
print(f"Trace link: https://platform.openai.com/logs/trace?trace_id={trace_id}\n\n")

with trace("Telling a joke", trace_id=trace_id):
    result = await Runner.run(gpt_agent, query)
    print(result.final_output)

From now on, to avoid lengthy code, we will not use custom trace id. We will use openAI default traces.


## Run agent in Streaming mode

Read more at https://openai.github.io/openai-agents-python/streaming/

In [ ]:
# Use Runner.run_streamed(agent, prompt) to run the agent in streaming mode
# Since it is an async function, we need to use 'await' to call it
# Iterate over the events from result.stream_events()
# For each event, if it is of type "raw_response_event" and its data is
# an instance of ResponseTextDeltaEvent, print the delta text as it arrives
# This will print the response in a streaming fashion

from openai.types.responses import ResponseTextDeltaEvent

gpt_agent = Agent(name="Clinical Expert", 
                instructions="You are a clinical expert in genetics", 
                model="gpt-4.1-nano")

query = "Tell me about ACMG classification of genetic variants"
result = Runner.run_streamed(gpt_agent, query)

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

From now on we will use non streaming mode for simplicity

## Interact with Agent with a chat interface
We can use Gradio chat for simple chat interface. 

Know more about Gradio at: https://www.gradio.app/

ChatInterface: https://www.gradio.app/docs/gradio/chatinterface


- Define chat function for Gradio interface
- Chat function takes message and history as input
- Pass history as context to the agent along with the message
- Get the final output from the agent's response

The below chat interface function won't have history. You have to explicitely pass message and history as query to 

In [ ]:
async def chat(message, history):
    # Uncomment below line to make chat interface know history
    # But better to use sqlite session for that
    # message = message + "Context: " + str(history)
    result = await Runner.run(gpt_agent, message)
    return result.final_output

import gradio as gr
gr.ChatInterface(
    chat,
    title="Clinical Expert Chatbot",
    description="Chat with the Clinical Expert"
).launch()


## Use session 

In [ ]:
from agents import SQLiteSession

# In memory session. This will not persist across restarts
session = SQLiteSession("my_session")

# Alternatively, you can use a file based session. This will persist across restarts
# session = SQLiteSession("my_session", "./my_session.db")


In [ ]:
async def chat(message, history):
    result = await Runner.run(gpt_agent, message, session=session)
    return result.final_output


import gradio as gr
gr.ChatInterface(
    chat,
    title="Clinical Expert Chatbot",
    description="Chat with the Clinical Expert"
).launch()

## Provide tools to agent

So far AI agents were linked with LLM models. Now we empower them with tools.

- Write a simple python function
- Provide docstring
- Decorate with @function_tool decorator

This way any python function is converted to Agent Tool.




#### Define Tools

We will define a function calculate_n_content, it returns total N bases + 10. Just to check agent uses tools. 


In [ ]:
from agents import function_tool

@function_tool
def calculate_n_content(dna_sequence: str) -> float:
    """Use this tool to calculate total N bases in a DNA sequence. 
    This is a custom tool which adds 10 to the N count.
    """
    n_count = dna_sequence.upper().count('N')
    return n_count+10

#### Define Agent

For tool calling, use some good model like gpt-4.1-mini which supports tool calling.
https://platform.openai.com/docs/models/gpt-4.1-mini 

In [ ]:
agent_instructions = """You are a helpful assistant for Bioinformatics task. 
You are equipped with calculate_n_content tool. 
If query involves DNA sequence N content analysis, you must use this tool to calculate N content. Otherwise chat normally.
"""

bioinfo_agent = Agent(name="BioinformaticsAgent",
                    instructions=agent_instructions,
                    model="gpt-4.1-mini",
                    tools=[calculate_n_content])

#### Run the Agent in a chat interface with session

In [ ]:
session = SQLiteSession("bioinfo_session")

async def chat(message, history):
    result = await Runner.run(bioinfo_agent, message, session=session)
    return result.final_output


import gradio as gr
gr.ChatInterface(
    chat,
    title="Bioinformatics Expert Chatbot",
    description="Chat with the Bioinformatics Expert"
).launch()

# You can ask below question to test the tool
# query = "N content in dna: ATGNNNATG"
